# Train Notebook
This notebook is used to perform various network training experiments. You can varing the hyperparameters related to your network in this notebook.

In addition to all previous test notebooks, we also need to complete all the functions given below, from **model.py**, before running this notebook.

Functions TO DO :

- [ ] Step 6a: Class Model  ➜ setup_model
- [ ] Step 6b: Class Model  ➜ train_step
- [ ] Step 6c: Class Model  ➜ validation_step
- [ ] Step 6d: Class Model  ➜ predict
- [ ] Step 6e: Class Model  ➜ get_best_model




In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import Dataset, ConcatDataset
from torchvision.transforms import Compose

from data.image_dataset import ImageDataset
from model import Model, get_device
from network.unet import UNet
from utils.experiment import Experiment
from utils.augmentation import VerticalFlip, Rotate, ColRec, GaussianBlur, GaussianNoise, ZoomIn, ZoomOut
from tqdm.autonotebook import tqdm


### 1. Create Experiment
Create a new experiment everytime you are training a new model.<br />
For __training__, set the argument __train =True__ <br />
For __predicting__ using a trained model, set the argument __new = False__ (See Predict notebook)

Give a meaningful name for the experiment, such that you can understand what parameters were used for training.
For example in the cell below, the experiment was named as 'Adam_w500' because we will be using the Adam optimiser and 1:500 weight ratio in the loss function.

When you create an experiment, the following experiment folders will be created:
1. data/network_data/Adam_w500 -> Stores the trained weights from each epoch as well as the experiment config file.
2. data/output_data/Adam_w500 -> Stores the video output containing predictions for each frame.
3. data/tensor_board_logs/Adam_w500 -> Stores the Tensorboard data generated during training



In [15]:
# Creating an experiment for training.
exp = Experiment(name='test23', train=True, overwrite=True)
cfg_path=exp.params['cfg_path']

### 2. Data Setup

In [16]:
# Train Set
augmentation_list_1 = Compose([
                                  VerticalFlip(probability=0.5),
                                  Rotate(probability=0.5),
                                  ColRec(probability=0.5),
                                  GaussianNoise(probability=0.5),
                                  GaussianBlur(probability=0.5),
                                  ZoomOut(probability=0.5),
                                  ZoomIn(probability=0.5),
                                 ])

augmentation_list_2 = Compose([
                                  VerticalFlip(probability=0.2),
                                  Rotate(probability=0.1),
                                  ColRec(probability=0.1),
                                  GaussianNoise(probability=0.1),
                                  GaussianBlur(probability=0.1),
                                  ZoomOut(probability=0.1),
                                  ZoomIn(probability=0.1),
                                 ])

augmentation_list_3 = Compose([
                                  VerticalFlip(probability=0.4),
                                  Rotate(probability=0.3),
                                  ColRec(probability=0.3),
                                  GaussianNoise(probability=0.3),
                                  GaussianBlur(probability=0.5),
                                  ZoomOut(probability=0.3),
                                  ZoomIn(probability=0.3),
                                 ])


train_dataset = ConcatDataset(
    [
        ImageDataset(dataset_name='train1',cfg_path=cfg_path,
                     is_train=True,augmentation=None),
        
        ImageDataset(dataset_name='train2',cfg_path=cfg_path,
                     is_train=True,augmentation=None),
        
        ImageDataset(dataset_name='train3',cfg_path=cfg_path,
                     is_train=True,augmentation=None),
        
        ImageDataset(dataset_name='CARLA_Train',cfg_path=cfg_path,
                     is_train=True,augmentation=None),
    
        ImageDataset(dataset_name='Real_No_Lane_Train',cfg_path=cfg_path, 
                     is_train=True,augmentation=None),

    ])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=6,
                                               shuffle=True, num_workers=0)

# Test Set
test_dataset =  ConcatDataset([ImageDataset(dataset_name='Sim_Lane_Test',size=20,cfg_path=cfg_path,is_train=False, seed=5), 
                              ImageDataset(dataset_name='CARLA_Test', cfg_path=cfg_path,is_train=False, seed=5)])

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1,
                                              shuffle=False, num_workers=4)

exp.update_configs()

### 3. Model Setup

In [17]:
# Initialize the model
model = Model(exp,net=UNet)

In [18]:
#Defime optimiser parameters
optimiser= torch.optim.Adam
optimiser_params = {'lr': 0.0002}


In [19]:
# Weighted Loss function
no_lane_weight = 1
lane_weight = 500

device = get_device()

# Non-lane is class 0 and lane is class 1
# loss function weight array -> [class_0_wt, class_1_wt]

weight = torch.FloatTensor([no_lane_weight, lane_weight]).to(device)
loss_func_params ={'weight' : weight}

In [20]:
model.setup_model(optimiser=optimiser,
                  optimiser_params=optimiser_params,
                  loss_function=torch.nn.CrossEntropyLoss,
                  loss_func_params=loss_func_params)

### 4. Training 

In [21]:
total_epochs = 500

print("\t\t Avg.Loss \t Accuracy \t Lane F1 Score")
model.reset_train_stats()

for i in range(1,total_epochs+1):

    print("Epoch [{}/{}]".format(i,total_epochs))

    model.net.train()
    # Train step
    for images,labels in tqdm(train_loader):
        model.train_step(images,labels)
    model.print_stats(mode= "Train")
    model.save_model(i)
    
    # Validation step
    model.net.eval()
    for image,labels in test_loader:
        model.validation_step(image,labels)
    model.print_stats(mode="Test")
    
    # After every 5 epochs, all saved models except the best model will
    # be deleted. This is because the storage space in student accounts 
    # in the lab is limited to 50GB.
    if i%5 == 0:
        model.cleanup()

# Final model cleanup incase total epochs is not divisible by 5
model.cleanup(final_step=True)


		 Avg.Loss 	 Accuracy 	 Lane F1 Score
Epoch [1/500]



Train 		  0.0768	   81.72%	   19.63%
Test 		  0.1293	   78.17%	   15.09%
Epoch [2/500]



Train 		  0.0479	   90.83%	   32.79%
Test 		  0.1017	   88.67%	   25.34%
Epoch [3/500]



Train 		  0.0321	   93.49%	   40.81%
Test 		  0.0773	   91.60%	   31.49%
Epoch [4/500]



Train 		  0.0297	   94.14%	   43.37%
Test 		  0.0789	   89.79%	   27.51%
Epoch [5/500]


...


Train 		  0.0013	   99.77%	   95.07%
Test 		  0.3518	   99.49%	   88.04%
Epoch [497/500]



Train 		  0.0008	   99.84%	   96.65%
Test 		  0.3879	   99.58%	   89.80%
Epoch [498/500]



Train 		  0.0007	   99.86%	   97.06%
Test 		  0.4736	   99.59%	   90.00%
Epoch [499/500]



Train 		  0.0006	   99.87%	   97.28%
Test 		  0.5245	   99.62%	   90.69%
Epoch [500/500]



Train 		  0.0006	   99.88%	   97.37%
Test 		  0.5021	   99.61%	   90.53%
Best model at Epoch 499 with F1 score 90.69%
